In [ ]:
# Adding system path
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))

In [ ]:
# Set to show warnings only once
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setting up displays
from IPython.display import display, HTML
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
## TODO : Need to install the matplotlib package/library
import matplotlib.pyplot as mat_plt
%matplotlib inline
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

In [ ]:
# Setting up autoreload for libs
%load_ext autoreload
%autoreload 2
%aimport optiml.queries

In [ ]:
# Initialize connection to Snowflake and set analysis date
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/kiva')
# Initialize query library
from optiml.queries import SNFLKQuery
qlib = SNFLKQuery(connection, 'KIV', cache_dir)
sdate = '2022-06-01'
edate = '2022-11-30'
print(f"The analysis is carried out for date range {sdate} to {edate}")

### WAREHOUSE ANALYSIS

In [ ]:
df = qlib.idle_warehouses(sdate, edate)
df = df.fillna('Unassigned')
df.head()

In [ ]:
trace1  = go.Scatter(
        mode='lines+markers',
        x = df['date_month'],
        y = df['name'],
        name="Warehouse Name",
        marker_color='crimson'
    )

data = [trace1]

layout = go.Layout(
    title_text='Idle Warehouses',
    yaxis=dict(
        title="Warehouse"
    ),
    xaxis=dict(
        title="Month"
    )
)
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
df = qlib.wh_scaled_up_out(sdate,edate,1)
df = df.fillna('Unassigned')
df.head()

In [ ]:
mat_plt.bar(df["warehouse_name"], df["spilling_read_ratio"])
mat_plt.xlabel("Warehouses")
mat_plt.ylabel("Spilling Ratio")
mat_plt.title("Warehouse required scale up")
mat_plt.show()

In [ ]:
df = qlib.wh_required_mcw()
df = df.fillna('Unassigned')
df.head()

In [ ]:
df = qlib.wh_under_utilization()
df = df.fillna('Unassigned')
df.head()